1. Создать Dataset ( https://www.kaggle.com/c/avito-demand-prediction) для загрузки данных (используем только числовые данные)

2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

*******************

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')
plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

********************************

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train/train.csv')
train.head(2)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000


In [ ]:
#  preprocessing
train.activation_date = pd.to_datetime(train.activation_date)
train['day_of_month'] = train.activation_date.apply(lambda x: x.day)
train['day_of_week'] = train.activation_date.apply(lambda x: x.weekday())

# кодирование средним
agg_cols = ['region', 'category_name',
            'city', 'user_type']
for col in tqdm_notebook(agg_cols):
    gp = train.groupby(col)['deal_probability']
    mean = gp.mean()
    train[col + '_deal_probability_avg'] = train[col].map(mean)

train = train.drop(['city', 'category_name', 'user_id', 'description', 'image', 'parent_category_name', 'region',
                    'item_id', 'param_1', 'param_2', 'param_3', 'title', 'user_type', 'activation_date'], axis=1)

for col in train.columns:
    if train[col].isna().sum() > 0:
        # train.drop(columns=col, inplace=True)
        train[col].fillna(train[col].median(), inplace=True)

display(train.head(2), train.info())

  0%|          | 0/4 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 10 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   price                               1503424 non-null  float64
 1   item_seq_number                     1503424 non-null  int64  
 2   image_top_1                         1503424 non-null  float64
 3   deal_probability                    1503424 non-null  float64
 4   day_of_month                        1503424 non-null  int64  
 5   day_of_week                         1503424 non-null  int64  
 6   region_deal_probability_avg         1503424 non-null  float64
 7   category_name_deal_probability_avg  1503424 non-null  float64
 8   city_deal_probability_avg           1503424 non-null  float64
 9   user_type_deal_probability_avg      1503424 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 114.7 MB


,price,item_seq_number,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg
0,400.0,2,1008.0,0.12789,28,1,0.122004,0.198445,0.123397,0.149557
1,3000.0,19,692.0,0.00000,26,6,0.136721,0.191848,0.139400,0.149557


None

In [ ]:
train, test = train_test_split(train, test_size=0.25, random_state=13)
train.head(2)

,price,item_seq_number,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg
31723,1300.0,6,1056.0,0.0,23,3,0.147066,0.178480,0.189069,0.149557
929954,1300.0,16,399.0,0.0,28,1,0.122004,0.046447,0.123397,0.149557


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127568 entries, 31723 to 1015882
Data columns (total 10 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   price                               1127568 non-null  float64
 1   item_seq_number                     1127568 non-null  int64  
 2   image_top_1                         1127568 non-null  float64
 3   deal_probability                    1127568 non-null  float64
 4   day_of_month                        1127568 non-null  int64  
 5   day_of_week                         1127568 non-null  int64  
 6   region_deal_probability_avg         1127568 non-null  float64
 7   category_name_deal_probability_avg  1127568 non-null  float64
 8   city_deal_probability_avg           1127568 non-null  float64
 9   user_type_deal_probability_avg      1127568 non-null  float64
dtypes: float64(7), int64(3)
memory usage: 94.6 MB


*********************************

In [ ]:
class DFDataset(torch.utils.data.Dataset):
    def __init__(self, df, normalize=False, fit_scaler=False):
        self.df = df.copy()
        self.normalize = normalize
        self.scaler = MinMaxScaler()
        self.fit_scaler = fit_scaler
        self.sc_fl = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx, -1:]
        if self.fit_scaler:
            self.scaler.fit(df.iloc[:, :-1])
            self.sc_fl = 1

        if self.normalize and sc_fl:
            df = scaler.transform(df)

        tensor = torch.FloatTensor(self.df.iloc[idx, 1:-1].values)
        label = torch.FloatTensor(label.values)

        return tensor, label

In [ ]:
train_dataset = DFDataset(train)
test_dataset = DFDataset(test)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2048,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=2048,
                                          shuffle=True,
                                          num_workers=4)

*****************************************

In [ ]:
class FFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FFNet, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 5*hidden_dim)
        self.dp1 = nn.Dropout(0.40)

        self.bn2 = nn.BatchNorm1d(5*hidden_dim)
        self.fc2 = nn.Linear(5*hidden_dim, 2*hidden_dim)
        self.dp2 = nn.Dropout(0.15)

        self.bn4 = nn.BatchNorm1d(2*hidden_dim)
        self.fc4 = nn.Linear(2*hidden_dim, 1)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.dp1(x)

        x = self.bn2(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.dp2(x)

        x = self.bn4(x)
        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

In [ ]:
def rmsle_loss(y_pred, y_true):
    loss = torch.sqrt(torch.mean((torch.log(y_pred+1)-torch.log(y_true+1))**2))
    return loss

In [ ]:
EPOCHES = 1
LR = 0.001

In [ ]:
def train_loop(tr_dataloader, ev_dataloader, model, optimizer, history):

    size = len(tr_dataloader.dataset)
    for batch, (X, y) in enumerate(tr_dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = rmsle_loss(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = batch * len(X)
            history['train'].append(loss)
            test_loss = eval_loop(ev_dataloader, model)
            history['eval'].append(test_loss)
            print(f"loss: {loss:>7f}, Avg test loss: {test_loss:>8f}  [{current:>5d}/{size:>5d}]")

    return history


def eval_loop(ev_dataloader, model):
    
    size = len(ev_dataloader.dataset)
    num_batches = len(ev_dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in ev_dataloader:
            pred = model(X.to(device))
            test_loss += rmsle_loss(pred, y.to(device))

    test_loss /= num_batches
    
    return test_loss

# SDG

In [ ]:
model = FFNet(8, 5).to(device)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [ ]:
history_1 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_1 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_1
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.306497, Avg test loss: 0.306702  [    0/1127568]
loss: 0.305421, Avg test loss: 0.305402  [102400/1127568]
loss: 0.303295, Avg test loss: 0.303989  [204800/1127568]
loss: 0.302771, Avg test loss: 0.302707  [307200/1127568]
loss: 0.300999, Avg test loss: 0.301348  [409600/1127568]
loss: 0.300000, Avg test loss: 0.299969  [512000/1127568]
loss: 0.297889, Avg test loss: 0.298604  [614400/1127568]
loss: 0.296901, Avg test loss: 0.297234  [716800/1127568]
loss: 0.296233, Avg test loss: 0.295913  [819200/1127568]
loss: 0.294895, Avg test loss: 0.294491  [921600/1127568]
loss: 0.293113, Avg test loss: 0.293072  [1024000/1127568]
loss: 0.292611, Avg test loss: 0.291642  [642400/1127568]
Done!


*************************

# RMSProp

In [ ]:
model = FFNet(8, 5).to(device)

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)

In [ ]:
history_2 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_2 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_2
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.320154, Avg test loss: 0.313924  [    0/1127568]
loss: 0.215786, Avg test loss: 0.215145  [102400/1127568]
loss: 0.133791, Avg test loss: 0.131552  [204800/1127568]
loss: 0.072291, Avg test loss: 0.069935  [307200/1127568]
loss: 0.042198, Avg test loss: 0.044369  [409600/1127568]
loss: 0.037592, Avg test loss: 0.038555  [512000/1127568]
loss: 0.035624, Avg test loss: 0.034550  [614400/1127568]
loss: 0.031134, Avg test loss: 0.030915  [716800/1127568]
loss: 0.026742, Avg test loss: 0.027238  [819200/1127568]
loss: 0.025252, Avg test loss: 0.024351  [921600/1127568]
loss: 0.021746, Avg test loss: 0.021720  [1024000/1127568]
loss: 0.019076, Avg test loss: 0.019650  [642400/1127568]
Done!


***********************************

# ADAM

In [ ]:
model = FFNet(8, 5).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
history_3 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_3 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_3
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.278434, Avg test loss: 0.277886  [    0/1127568]
loss: 0.246901, Avg test loss: 0.247053  [102400/1127568]
loss: 0.212971, Avg test loss: 0.212828  [204800/1127568]
loss: 0.162389, Avg test loss: 0.162362  [307200/1127568]
loss: 0.103534, Avg test loss: 0.102612  [409600/1127568]


Вывод: удалось проверить скорость схождения при фиксированной скорости обучения: быстрее всего RMSProp, медленее - SGD. Для проверки качества нехватило ресурсов collab